![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import statsmodels as sm
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf, grangercausalitytests
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics import api as smg

qb = QuantBook()
start_time = datetime(2020, 1, 1) # start datetime for history call
end_time = datetime(2020, 12, 1) # end datetime for history call

In [ ]:
    #Dickey-Fuller test stolen from https://www.hackdeploy.com/augmented-dickey-fuller-test-in-python/
    def ADF_Stationarity_Test(timeseries, significance_level=0.05, printResults = True):
        is_stationary = None
        
        adf_test = adfuller(timeseries, autolag='AIC')
        pValue = adf_test[1]
        
        if (pValue<significance_level):
            is_stationary = True
        else:
            is_stationary = False
        
        if printResults:
            dfResults = pd.Series(adf_test[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
            #Add Critical Values
            for key,value in adf_test[4].items():
                dfResults['Critical Value (%s)'%key] = value
            print('Augmented Dickey-Fuller Test Results:')
            print(dfResults)
            print('Is it stationary? '+str(is_stationary))
        else:
            return is_stationary

In [ ]:
oil = qb.AddCfd("WTICOUSD", Resolution.Daily, Market.Oanda)
oil_data = qb.History(["WTICOUSD"], 4*365, Resolution.Daily)
print(oil_data['close'].head())
print(oil_data['close'].tail())
print(oil_data['close'].size)
oil_data['close'].plot(title='Crude Oil')

In [ ]:
print('Running Augmented Dickey-Fuller Test for Crude Oil...\n')
ADF_Stationarity_Test(oil_data['close'].values)

In [ ]:
# create a differenced series
def difference(series, interval=1):
    diff = []
    for i in range(interval, len(series)):
        value = series[i] - series[i - interval]
        diff.append(value)
    return diff

In [ ]:
print('Making data stationary...\n')
stationary_oil_data = pd.DataFrame(difference(oil_data['close']))
stationary_oil_data.plot(title='Stationary Crude Oil')

In [ ]:
print('Running Augmented Dickey-Fuller Test for Stationary Crude Oil...\n')
ADF_Stationarity_Test(stationary_oil_data.values)

In [ ]:
# Auto correlation test on raw data
auto_corr_oil = plot_acf(oil_data['close'].values)
auto_corr_oil.show()

In [ ]:
# Partial auto correlation test on raw data
part_auto_corr = plot_pacf(oil_data['close'].values)
part_auto_corr.show()

In [ ]:
print('Checking ARIMA(1,1,6)')

model = sm.tsa.arima_model.ARIMA(oil_data['close'].values, order=(1, 1, 6))
res = model.fit()
plt.plot(stationary_oil_data)
plt.plot(res.fittedvalues*8, color='red')

In [ ]:

# The following is the output of some locally written code as it took too long on QCBook:
"""
Testing Predictions for 292 days.

Testing ARIMA (1, 1, 6)
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 
MSE = 1.6108850135912554
"""
MSE = 1.6108850135912554
RMSE = math.sqrt(MSE)
print('MSE  = ' + str(MSE))
print('RMSE = ' + str(RMSE))